In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, LinearColorMapper, HoverTool
from bokeh.palettes import Greys3
from bokeh.models import FactorRange
from bokeh.models import Range1d, RangeSlider, CustomJS
from bokeh.layouts import column

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def load_inputs(npz_path, ps4g_path, sep):
    data = np.load(npz_path, allow_pickle=True)
    final_predictions = data['path']
    background = data['background']
    model_predictions = data['predictions']

    df = pd.read_csv(ps4g_path, sep=sep, header=0)
    gamete = df['gamete'].values
    index = df['gamete_index'].values
    y_labels = list(gamete[index])

    return final_predictions, background, model_predictions, y_labels

In [4]:
npz_path  = "/Users/bm646-admin/Downloads/HMM_CML69_9_100.npz"
ps4g_path = "/Users/bm646-admin/Downloads/CML69.csv"

In [5]:
fp, bg, mp, ylab = load_inputs(npz_path, ps4g_path, sep = "\t")

In [6]:
arr = bg.transpose()[:, 0:1000]

In [7]:
ylab.append("XSample")
n_rows, n_cols = arr.shape

# Build a DataFrame where each row is one cell
df = pd.DataFrame({
    'row': np.repeat(np.arange(n_rows), n_cols),
    'col': np.tile(np.arange(n_cols), n_rows),
    'value': arr.ravel(),
})

df['row_label'] = df['row'].map(lambda i: ylab[i])

In [8]:
source = ColumnDataSource(df)
mapper = LinearColorMapper(palette = Greys3, low=0, high=1)

In [9]:
factors = ylab[::-1]

p = figure(
    title   = "Imputation visualizer",
    x_range = Range1d(start=0, end=n_cols, bounds=(0, n_cols)), 
    y_range = FactorRange(*factors),
    tools   = "hover,box_zoom,reset,save",
    x_axis_location = "above",
    tooltips = [("Gamete ", "@row_label"), ("Position ", "@col"), ("NP 'background' val ", "@value")]
)

In [10]:
p.rect(
    x = 'col', y = 'row_label',
    width = 1, height = 1,
    source     = source,
    fill_color = {'field': 'value', 'transform': mapper},
    line_color = "#fff"
)

GlyphRenderer(id='p1048', ...)

In [11]:
p.width = 1000 
p.height = 500
p.grid.grid_line_color = None 
p.axis.axis_line_color = None 
p.axis.major_tick_line_color = None 
p.axis.major_label_text_font_size = "7px" 
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

In [14]:
slider = RangeSlider(start=0, end=n_cols, value=(0,19), step=1, title="Range selector")
slider.js_on_change('value', CustomJS(args=dict(xr=p.x_range), code="""
    xr.start = cb_obj.value[0];
    xr.end   = cb_obj.value[1];
"""))

In [18]:
layout = column(slider, p)
show(layout)